# Market Return Analysis

Ananya Mathu

In [112]:
import pandas as pd
import numpy as np
import yfinance as yf
#great yahoo finance api
import os
import datetime
import time
import matplotlib.pyplot as plt
import wrds
#create a gz file so that you don't need to keep putting password for wrds, 
#wharton research data survey main provider for securities prices for academics and non-academics also
from pandas.tseries.offsets import *
# Settings: shares code, exchange codes to use, sample period, folder for pictures/temporary data/outout
use_NYSE_brkpts = False

Construct the value-weighted market return using CRSP data,1 replicating the market return
time series available in Kenneth French website. 

Also calculate the equal-weighted market return,
and the lagged total market capitalization. The output we consider is from January 1926 to December
2022, at a monthly frequency.

In [113]:
# read the CSV file into a Pandas DataFrame
df = pd.read_csv("C:/Users/anany/Downloads/returns.csv")

# print the first five rows of the DataFrame
print(df.head())

    permno        date       ret  shrout     prc  shrcd  exchcd
0  10000.0  1985-12-31       NaN     NaN     NaN    NaN     NaN
1  10000.0  1986-01-31       NaN  3680.0 -4.3750   10.0     3.0
2  10000.0  1986-02-28 -0.257143  3680.0 -3.2500   10.0     3.0
3  10000.0  1986-03-31  0.365385  3680.0 -4.4375   10.0     3.0
4  10000.0  1986-04-30 -0.098592  3793.0 -4.0000   10.0     3.0


In [114]:
#Cleaning the data
# Remove any null or missing values
df = df.dropna()
# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# Sort the dataframe by the date column
df = df.sort_values(by='date')
# Print the cleaned and sorted dataframe
df.head()

,permno,date,ret,shrout,prc,shrcd,exchcd
428699,12845.0,1926-01-30,0.076149,860.0,-93.625,10.0,1.0
226310,11420.0,1926-01-30,0.086957,1297.0,3.125,10.0,1.0
398278,12570.0,1926-01-30,0.069215,394.0,129.375,11.0,1.0
1559541,27692.0,1926-01-30,0.000000,77.0,45.000,10.0,1.0
573597,14146.0,1926-01-30,-0.020619,3077.0,11.875,10.0,1.0


In [115]:
f_df = pd.read_csv("C:/Users/anany/Downloads/F-F_Research_Data_Factors_daily_CSV/F-F_Research_Data_Factors_daily.CSV", skiprows = 4) 

In [116]:
f_df.head()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,19260701,0.10,-0.25,-0.27,0.009
1,19260702,0.45,-0.33,-0.06,0.009
2,19260706,0.17,0.30,-0.39,0.009
3,19260707,0.09,-0.58,0.02,0.009
4,19260708,0.21,-0.38,0.19,0.009


In [117]:
f_df = f_df.rename(columns={f_df.columns[0]: 'date'})
f_df.head()

,date,Mkt-RF,SMB,HML,RF
0,19260701,0.10,-0.25,-0.27,0.009
1,19260702,0.45,-0.33,-0.06,0.009
2,19260706,0.17,0.30,-0.39,0.009
3,19260707,0.09,-0.58,0.02,0.009
4,19260708,0.21,-0.38,0.19,0.009


In [118]:
# Remove rows that contain text
f_df = f_df[f_df['date'].str.isdigit()]

# Convert date column to datetime format
f_df['date'] = pd.to_datetime(f_df['date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
f_df.head()

,date,Mkt-RF,SMB,HML,RF
0,1926-07-01,0.10,-0.25,-0.27,0.009
1,1926-07-02,0.45,-0.33,-0.06,0.009
2,1926-07-06,0.17,0.30,-0.39,0.009
3,1926-07-07,0.09,-0.58,0.02,0.009
4,1926-07-08,0.21,-0.38,0.19,0.009


In [129]:
f_df['date'] = pd.to_datetime(f_df['date'])
merged_df = pd.merge(f_df, df, on='date')

In [130]:
merged_df['ret'] = pd.to_numeric(merged_df['ret'], errors='coerce')

In [140]:
# Step 1: Calculate market value of each stock
merged_df['MktCap'] = merged_df['shrout'] * merged_df['prc']

# Step 2: Calculate total market value of portfolio on each day
port_mktcap = merged_df.groupby('date')['MktCap'].sum()

# Step 3: Calculate weight of each stock in portfolio
merged_df['Weight'] = merged_df['MktCap'] / port_mktcap[merged_df['date']].values

# Step 4: Calculate return of each stock on each day
merged_df['AdjRet'] = (1 + merged_df['ret']) * (1 + merged_df['ret']) - 1

# Step 5: Calculate value-weighted index return
vw_index = (merged_df['Weight'] * merged_df['AdjRet']).groupby(merged_df['date']).sum()
merged_df.head()

,date,Mkt-RF,SMB,HML,RF,permno,ret,shrout,prc,shrcd,exchcd,MktCap,Weight,AdjRet,EqualWeight,Ret
0,1926-07-31,0.46,-0.12,-0.17,0.009,12984.0,0.000000,676.0,16.00,10.0,1.0,10816.0,0.000520,0.000000,2.111912e-07,0.000000
1,1926-07-31,0.46,-0.12,-0.17,0.009,10380.0,0.130000,200.0,-56.50,10.0,1.0,-11300.0,-0.000543,0.276900,2.111912e-07,0.276900
2,1926-07-31,0.46,-0.12,-0.17,0.009,10823.0,0.169753,508.0,94.75,11.0,1.0,48133.0,0.002312,0.368322,2.111912e-07,0.368322
3,1926-07-31,0.46,-0.12,-0.17,0.009,11519.0,0.194030,30.0,-30.00,10.0,1.0,-900.0,-0.000043,0.425707,2.111912e-07,0.425707
4,1926-07-31,0.46,-0.12,-0.17,0.009,11455.0,-0.005747,400.0,129.75,10.0,1.0,51900.0,0.002493,-0.011461,2.111912e-07,-0.011461


In [132]:
vw_index

date
1926-07-31    0.098324
1926-08-31    0.071793
1926-09-30    0.027531
1926-10-30   -0.057152
1926-11-30    0.071861
                ...   
2022-08-31   -0.055177
2022-09-30   -0.160974
2022-10-31    0.189156
2022-11-30    0.138504
2022-12-30   -0.091208
Length: 1158, dtype: float64

In [135]:
# Step 2: Calculate equal weight of each stock in portfolio
merged_df['EqualWeight'] = 1 / len(merged_df)

# Step 3: Calculate total market value of portfolio on each day
port_mktcap = (merged_df['MktCap'] * merged_df['EqualWeight']).groupby(merged_df['date']).sum()

# Step 4: Calculate lagged total market capitalization
lag_port_mktcap = port_mktcap.shift()

# Step 5: Calculate return of each stock on each day
merged_df['Ret'] = (1 + merged_df['ret']) * (1 + merged_df['ret']) - 1

# Step 6: Calculate equal-weighted index return
eq_index = (merged_df['EqualWeight'] * merged_df['Ret']).groupby(merged_df['date']).sum()

# Print output
print('Equal-weighted market return from January 1926 to December 2022:')
print(eq_index.loc['1926-01-31':'2022-12-31'])

print('\nLagged total market capitalization from January 1926 to December 2022:')
print(lag_port_mktcap.loc['1926-01-31':'2022-12-31'])

Equal-weighted market return from January 1926 to December 2022:
date
1926-07-31    3.642569e-06
1926-08-31    8.261289e-06
1926-09-30   -5.065919e-07
1926-10-30   -6.378778e-06
1926-11-30    6.986916e-06
                  ...     
2022-08-31   -3.312700e-06
2022-09-30   -3.366014e-04
2022-10-31    2.643538e-04
2022-11-30    1.877437e-04
2022-12-30   -1.082866e-04
Length: 1158, dtype: float64

Lagged total market capitalization from January 1926 to December 2022:
date
1926-07-31             NaN
1926-08-31        4.396385
1926-09-30        5.176561
1926-10-30        5.152285
1926-11-30        3.781404
                  ...     
2022-08-31    11661.331679
2022-09-30    11218.064874
2022-10-31    10163.475831
2022-11-30    10903.285768
2022-12-30    11486.556427
Length: 1158, dtype: float64


In [136]:
from scipy.stats import skew, kurtosis

Using the risk-free rate of return from French’s website, report the following moments of
the market excess returns for both time series (chose 4 decimal digits): annualized return, annualized
volatility, annualized Sharpe ratio, skewness, and excess kurtosis. 

Comparing between July 1926 to December 2022, at a monthly frequency.

In [137]:
excess_returns = merged_df['Mkt-RF']
# Calculate annualized moments
annualized_return = np.power((1 + excess_returns.mean()), 12) - 1
annualized_volatility = excess_returns.std() * np.sqrt(12)
sharpe_ratio = annualized_return / annualized_volatility

# Calculate skewness and excess kurtosis
skewness = skew(excess_returns)
excess_kurtosis = kurtosis(excess_returns) - 3

# Print the results
print(f'Annualized Return: {annualized_return:.4f}')
print(f'Annualized Volatility: {annualized_volatility:.4f}')
print(f'Sharpe Ratio: {sharpe_ratio:.4f}')
print(f'Skewness: {skewness:.4f}')
print(f'Excess Kurtosis: {excess_kurtosis:.4f}')

Annualized Return: 1.1773
Annualized Volatility: 3.3937
Sharpe Ratio: 0.3469
Skewness: -0.4522
Excess Kurtosis: 4.3948


Reporting (up to 8 decimal digits) the correlation between our time series and French’s time
series, and the maximum absolute difference between the two time series. 

In [141]:
# Compute correlation and maximum absolute difference
corr = merged_df[['ret', 'Mkt-RF', 'SMB', 'HML']].corr().iloc[0, 1:]
max_diff = (merged_df[['ret', 'Mkt-RF', 'SMB', 'HML']].abs().max() - merged_df[['ret', 'Mkt-RF', 'SMB', 'HML']].abs().min()).max()

print(f"Correlation: {corr.max():.8f}")
print(f"Maximum absolute difference: {max_diff:.8f}")

Correlation: 0.02286258
Maximum absolute difference: 24.00000000


The maximum absolute difference between the two time series is 24, which is not economically negligible. The difference could be due to several reasons. One reason could be that the CRSP data includes all listed companies in the US, while the Fama-French data only includes a subset of companies based on certain criteria. Another reason could be that the two data sources may use different methodologies to calculate the returns, market factors, and other variables. Additionally, the CRSP data includes delisted companies, which may have a different impact on the returns and other variables. Overall, while the correlation between the two time series is low, the difference may not be significant enough to affect the results of most analyses. However, it is important to be aware of these differences and to carefully consider them when interpreting the results of any analysis.